In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 로드
df = pd.read_csv('train_real_final.csv')

# 2. 특성과 타겟 분리
X = df.drop(columns=['y'])
y = df['y']

# 3. 8:2 분할 (train vs test)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


# 4. 커스텀 스코어 함수 정의
def mean_score_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    if hasattr(estimator, "predict_proba"):
        y_proba = estimator.predict_proba(X)
        if y_proba.shape[1] == 2:
            auc = roc_auc_score(y, y_proba[:, 1])
        else:
            auc = roc_auc_score(y, y_proba, multi_class='ovr')
    else:
        auc = 0  # 예외 처리 (예: 확률 없음)
    acc = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='weighted')
    return np.mean([acc, f1, auc])


# 5. 1차 Random Search
param_dist = {
    'n_estimators': np.arange(100, 501, 100),  # 100, 200, 300, 400, 500
    'max_depth': [10, 20, 30, None],           # 적당한 깊이 + 제한 없는 경우
    'min_samples_split': [2, 5, 10],           # 기본값 + 과적합 방지
    'min_samples_leaf': [1, 2, 4],             # 리프 노드 최소 샘플 수
    'max_features': ['sqrt', 'log2']           # 고차원 원핫에 맞게 자동 설정
}


# 6. 랜덤포레스트 모델 정의
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# 7. 1차 랜덤서치 정의
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring=mean_score_scorer,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    verbose=2,
    random_state=42,
    n_jobs=-1,
    error_score='raise'
)

random_search.fit(X_train, y_train)

# 8. 결과 출력
print("Best Parameters:", random_search.best_params_)
print("Best Mean Score:", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters: {'n_estimators': np.int64(300), 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20}
Best Mean Score: 0.6780066462037551


In [4]:
from sklearn.model_selection import GridSearchCV

# 1. 이전 랜덤서치 결과 기반의 그리드 탐색 범위 설정
param_grid = {
    'n_estimators': [250, 300, 350],
    'max_depth': [15, 20, 25],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [1, 2],
    'max_features': ['log2']
}

# 2. 모델 정의 (같은 설정 유지)
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# 3. 그리드서치 객체 생성
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mean_score_scorer,  # 이전에 정의한 mean_score_scorer 기반
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    verbose=2,
    n_jobs=-1,
    error_score='raise'
)

# 4. 트레인셋 기반 튜닝
grid_search.fit(X_train, y_train)

# 5. 결과 출력
print("Best Parameters (GridSearch):", grid_search.best_params_)
print("Best Mean Score (GridSearch):", grid_search.best_score_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters (GridSearch): {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 250}
Best Mean Score (GridSearch): 0.6784276150195074


In [5]:
# 5. 최적 파라미터로 모델 생성 및 훈련
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# 6. 검증셋으로 예측
y_val_pred = best_rf.predict(X_val)
y_val_proba = best_rf.predict_proba(X_val)

# 7. 성능 평가
acc = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, average='binary')
roc_auc = roc_auc_score(y_val, y_val_proba[:, 1])  # 클래스 1 확률 사용

# 평균 점수 계산
mean_score = np.mean([acc, f1, roc_auc])

# 8. 결과 출력
print("Accuracy:", acc)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Mean Score:", mean_score)

Accuracy: 0.6500798904359735
F1 Score: 0.6442330006962173
ROC AUC Score: 0.7060298658770016
Mean Score: 0.6667809190030641
